In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!ln -s /content/gdrive/My\ Drive/ /mydrive

In [17]:
%cd /mydrive/yolov4/darknet

/content/gdrive/My Drive/yolov4/darknet


In [18]:
!chmod +x ./darknet

In [5]:
%pip install paho-mqtt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 5.6 MB/s eta 0:00:00


In [6]:
!sudo apt-get install mosquitto mosquitto-clients

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcjson1 libdlt2 libev4 libmosquitto1 libwebsockets16 libwrap0
Suggested packages:
  apparmor
The following NEW packages will be installed:
  libcjson1 libdlt2 libev4 libmosquitto1 libwebsockets16 libwrap0 mosquitto
  mosquitto-clients
0 upgraded, 8 newly installed, 0 to remove and 45 not upgraded.
Need to get 696 kB of archives.
After this operation, 2,079 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcjson1 amd64 1.7.15-1 [15.5 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libdlt2 amd64 2.18.6-2 [52.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libmosquitto1 amd64 2.0.11-1ubuntu1.1 [51.8 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libev4 amd64 1:4.33-1 [29.4 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/univ

In [19]:
# difference+MQTT
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import paho.mqtt.client as mqtt
import cv2
import numpy as np
import PIL
import io
import html
import time
import json
import matplotlib.pyplot as plt
%matplotlib inline

from darknet import *

# YOLO model setup
cfg_path = 'cfg/yolov4-custom.cfg'
data_path = 'data/obj.data'
weights_path = '/mydrive/yolov4/training/yolov4-custom_best.weights'
network, class_names, class_colors = load_network(cfg_path, data_path, weights_path)
width = network_width(network)
height = network_height(network)

# MQTT 브로커 주소 (공용 MQTT 브로커 사용)
broker_address = "broker.hivemq.com"  # 공용 브로커 주소
port = 1883
topic = "tuk/motor"
topic1 = "tuk/control"
topic2 = "tuk/mode"

# topic2에서 받은 메시지를 저장할 변수
received_mode_message = ""

# 메시지가 도착했을 때 호출되는 콜백 함수
def on_message(client, userdata, message):
    global received_mode_message
    if message.topic == topic2:
        received_mode_message = str(message.payload.decode('utf-8'))

# MQTT 클라이언트 생성 및 설정
client = mqtt.Client()
client.on_message = on_message

# 브로커에 연결
client.connect(broker_address, port=port)

# 구독 설정
client.subscribe(topic2)

# 메시지 수신 대기
client.loop_start()

# darknet helper function to run detection on image
def darknet_helper(img, width, height):
    darknet_image = make_image(width, height, 3)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (width, height), interpolation=cv2.INTER_LINEAR)

    # get image ratios to convert bounding boxes to proper size
    img_height, img_width, _ = img.shape
    width_ratio = img_width / width
    height_ratio = img_height / height

    # run model on darknet style image to get detections
    copy_image_from_bytes(darknet_image, img_resized.tobytes())
    detections = detect_image(network, class_names, darknet_image)
    free_image(darknet_image)
    return detections, width_ratio, height_ratio

# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
    image_bytes = b64decode(js_reply.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string
def bbox_to_bytes(bbox_array):
    bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
    iobuf = io.BytesIO()
    bbox_PIL.save(iobuf, format='png')
    bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))
    return bbox_bytes

# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
    js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
        stream.getVideoTracks()[0].stop();
        video.remove();
        div.remove();
        video = null;
        div = null;
        stream = null;
        imgElement = null;
        captureCanvas = null;
        labelElement = null;
    }

    function onAnimationFrame() {
        if (!shutdown) {
            window.requestAnimationFrame(onAnimationFrame);
        }
        if (pendingResolve) {
            var result = "";
            if (!shutdown) {
                captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
                result = captureCanvas.toDataURL('image/jpeg', 0.8)
            }
            var lp = pendingResolve;
            pendingResolve = null;
            lp(result);
        }
    }

    async function createDom() {
        if (div !== null) {
            return stream;
        }
        div = document.createElement('div');
        div.style.border = '2px solid black';
        div.style.padding = '3px';
        div.style.width = '100%';
        div.style.maxWidth = '600px';
        document.body.appendChild(div);

        const modelOut = document.createElement('div');
        modelOut.innerHTML = "<span>Status:</span>";
        labelElement = document.createElement('span');
        labelElement.innerText = 'No data';
        labelElement.style.fontWeight = 'bold';
        modelOut.appendChild(labelElement);
        div.appendChild(modelOut);

        video = document.createElement('video');
        video.style.display = 'block';
        video.width = div.clientWidth - 6;
        video.setAttribute('playsinline', '');
        video.onclick = () => { shutdown = true; };
        stream = await navigator.mediaDevices.getUserMedia(
            {video: { facingMode: "environment"}});
        div.appendChild(video);
        imgElement = document.createElement('img');
        imgElement.style.position = 'absolute';
        imgElement.style.zIndex = 1;
        imgElement.onclick = () => { shutdown = true; };
        div.appendChild(imgElement);

        const instruction = document.createElement('div');
        instruction.innerHTML =
            '<span style="color: red; font-weight: bold;">' +
            'When finished, click here or on the video to stop this demo</span>';
        div.appendChild(instruction);
        instruction.onclick = () => { shutdown = true; };

        video.srcObject = stream;
        await video.play();
        captureCanvas = document.createElement('canvas');
        captureCanvas.width = 640;
        captureCanvas.height = 480;
        window.requestAnimationFrame(onAnimationFrame);

        return stream;
    }
    async function stream_frame(label, imgData) {
        if (shutdown) {
            removeDom();
            shutdown = false;
            return '';
        }
        var preCreate = Date.now();
        stream = await createDom();

        var preShow = Date.now();
        if (label != "") {
            labelElement.innerHTML = label;
        }

        if (imgData != "") {
            var videoRect = video.getClientRects()[0];
            imgElement.style.top = videoRect.top + "px";
            imgElement.style.left = videoRect.left + "px";
            imgElement.style.width = videoRect.width + "px";
            imgElement.style.height = videoRect.height + "px";
            imgElement.src = imgData;
        }

        var preCapture = Date.now();
        var result = await new Promise(function(resolve, reject) {
            pendingResolve = resolve;
        });
        shutdown = false;

        return {'create': preShow - preCreate,
                'show': preCapture - preShow,
                'capture': Date.now() - preCapture,
                'img': result};
    }
    ''')

    display(js)

def video_frame(label, bbox):
    data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
    return data

# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialize bounding box to empty
bbox = ''
a = 0
b = 0

while True:

    # topic1에 screen 메시지 전송
    if received_mode_message == "cus":
      b=b+1
      if b==1:
        client.publish(topic1, "screen")
        b=0
      #print(b)
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    frame = js_to_image(js_reply["img"])

    # create transparent overlays for bounding boxes
    bbox_array = np.zeros([480, 640, 4], dtype=np.uint8)
    # 추가 바운딩 박스를 위한 새로운 변수 생성
    additional_bbox_array = np.zeros([480, 640, 4], dtype=np.uint8)

    # call our darknet helper on video frame
    detections, width_ratio, height_ratio = darknet_helper(frame, width, height)

    for label, confidence, bbox in detections:
        left, top, right, bottom = bbox2points(bbox)
        left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)

        # 바운딩 박스를 10 픽셀씩 축소
        padding_rl = 70
        padding_u = 200
        padding_d = 100
        img_height, img_width, _ = frame.shape
        left = max(0, left + padding_rl)
        top = max(0, top + padding_u)
        right = min(img_width, right - padding_rl)
        bottom = min(img_height, bottom - padding_d)

        if float(confidence) > 90:
          # Create a bounding box
          bbox_array = cv2.rectangle(bbox_array, (left, top), (right, bottom), class_colors[label], 2)
          bbox_array = cv2.putText(bbox_array, "{} [{:.2f}]".format(label, float(confidence)), (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, class_colors[label], 2)
          a += 1
        else :
          #print("stop")
          client.publish(topic, "stop")
          client.publish(topic, json.dumps([0, 0, 0, 0]))

    '''
    if a == 3:
        Add_Box_left = left
        Add_Box_top = top
        Add_Box_right = right
        Add_Box_bottom = bottom
    '''

    Add_Box_left = 0
    Add_Box_top = 162
    Add_Box_right = 640
    Add_Box_bottom = 428


    # additional_bbox_array 바운딩 박스 추가
    additional_bbox_array = cv2.rectangle(additional_bbox_array, (Add_Box_left, Add_Box_top), (Add_Box_right, Add_Box_bottom), (255, 0, 0, 255), 2)
    #additional_bbox_array = cv2.rectangle(additional_bbox_array, (0,162),(640,428), (255, 0, 0, 255), 2)
    additional_bbox_array = cv2.putText(additional_bbox_array, "Additional Box", (100, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0, 255), 2)

    box_width = right - left
    box_height = bottom - top
    box_middle_dot_x = left + box_width // 2
    box_middle_dot_y = top + box_height // 2
    box_area = box_width * box_height
    stadard_width = Add_Box_right - Add_Box_left
    stadard_height = Add_Box_bottom - Add_Box_top
    stadard_middle_dot_x = Add_Box_left + stadard_width // 2
    stadard_middle_dot_y = Add_Box_top + stadard_height // 2
    stadard_area = stadard_width * stadard_height
    left = stadard_middle_dot_x - box_middle_dot_x
    right = box_middle_dot_x - stadard_middle_dot_x

    # 메시지 수신 대기
    client.loop_start()

    # topic2에서 받은 메시지가 'cus'인지 확인
    if received_mode_message == "cus":
        # 바운딩 박스 크기 비교
        if stadard_area > box_area:
            if 0 <= abs(box_middle_dot_x - stadard_middle_dot_x) <= 100:
                #print("forward")
                client.publish(topic, "forward")
                client.publish(topic, json.dumps([0, 0, 1, 0]))
            elif 100 < stadard_middle_dot_x - box_middle_dot_x:
                #print("left")
                client.publish(topic, "left")
                client.publish(topic, json.dumps([0, 0, 0, 1]))
            elif 100 < box_middle_dot_x - stadard_middle_dot_x:
                #print("right")
                client.publish(topic, "right")
                client.publish(topic, json.dumps([0, 0, 1, 1]))
        else:
            #print("stop")
            client.publish(topic, "stop")
            client.publish(topic, json.dumps([0, 0, 0, 0]))

    # combine both bounding box arrays
    combined_bbox_array = cv2.addWeighted(bbox_array, 1.0, additional_bbox_array, 1.0, 0)
    combined_bbox_array[:, :, 3] = (combined_bbox_array.max(axis=2) > 0).astype(int) * 255

    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(combined_bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes


OSError: /content/gdrive/My Drive/yolov4/darknet/libdarknet.so: cannot open shared object file: No such file or directory

In [13]:
# Simple
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import matplotlib.pyplot as plt
%matplotlib inline


# import darknet functions to perform object detections
from darknet import *
# load in our YOLOv4 architecture network
network, class_names, class_colors = load_network("cfg/yolov4-custom.cfg", "data/obj.data", "/mydrive/yolov4/training/yolov4-custom_best.weights")
width = network_width(network)
height = network_height(network)

# darknet helper function to run detection on image
def darknet_helper(img, width, height):
  darknet_image = make_image(width, height, 3)
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img_resized = cv2.resize(img_rgb, (width, height),
                              interpolation=cv2.INTER_LINEAR)

  # get image ratios to convert bounding boxes to proper size
  img_height, img_width, _ = img.shape
  width_ratio = img_width/width
  height_ratio = img_height/height

  # run model on darknet style image to get detections
  copy_image_from_bytes(darknet_image, img_resized.tobytes())
  detections = detect_image(network, class_names, darknet_image)
  free_image(darknet_image)
  return detections, width_ratio, height_ratio

# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }
      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);
      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();
      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }
      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    frame = js_to_image(js_reply["img"])

    # 좌우반전 수행
    #frame = cv2.flip(frame, 1)

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)


    # call our darknet helper on video frame
    detections, width_ratio, height_ratio = darknet_helper(frame, width, height)

    # loop through detections and draw them on transparent overlay image
    for label, confidence, bbox in detections:
      left, top, right, bottom = bbox2points(bbox)
      left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)

      # 여기서 바운딩 박스를 10 픽셀씩 축소
      padding_rl = 70
      padding_u = 200
      padding_d = 100
      img_height, img_width, _ = frame.shape
      left = max(0, left + padding_rl)
      top = max(0, top + padding_u)
      right = min(img_width, right - padding_rl)
      bottom = min(img_height, bottom - padding_d)

      #bbox_array = cv2.rectangle(bbox_array, (left, top), (right, bottom), class_colors[label], 2)
      bbox_array = cv2.rectangle(bbox_array, (left, top), (right, bottom), class_colors[label], 2)
      bbox_array = cv2.putText(bbox_array, "{} [{:.2f}]".format(label, float(confidence)),(left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,class_colors[label], 2)
      #bbox_array = cv2.putText(bbox_array, "a",(left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,class_colors[label], 2)

    # (100, 100) 위치에 50x50 바운딩 박스 추가
    bbox_array = cv2.rectangle(bbox_array, (145, 170), (448, 417), (255, 0, 0, 255), 2)
    bbox_array = cv2.putText(bbox_array, "Additional Box", (100, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0, 255), 2)

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes


OSError: /content/gdrive/MyDrive/yolov4/darknet/libdarknet.so: cannot open shared object file: No such file or directory